In [13]:
import warnings
import agate
from pathlib import Path
from shutil import copyfile
import os
import csv
from decimal import *

warnings.filterwarnings(action='ignore')

g_table_geos = agate.Table.from_csv('../data/county-geos.csv')

def join_tables(fp, classify_column,  columns_i):            
    try:
        table = agate.Table.from_csv(fp, encoding='utf-8-sig')
        
        # Not our first time through
        # Join tables
        if Path(g_table_merge_fp).is_file():
            print("file exists")
            # Take our global table
            # And our table getting passed through the function
            # give them both geo ids
            # then join to make one big, beautiful table
            table_one = agate.Table.from_csv(g_table_merge_fp, encoding='utf-8-sig')
            table_geos = table_one.join(g_table_geos, classify_column, 'county',
                                    inner=True,
                                    columns=columns_i)
            
            table_one.to_csv('../tmp/table-one.csv')
            table_geos.to_csv('../tmp/table-geos.csv')
            
            table_geos_two = table.join(g_table_geos, classify_column, 'county',
                                    inner=True,
                                    columns=columns_i)
            table_join = table_geos.join(table_geos_two, 'geo', 'geo')
            
            table_geos_two.to_csv('../tmp/table-geos-two.csv')
            table_join.to_csv('../tmp/table-join.csv')
                
            # If something didn't join right, print it out
            if table_join:
                table_join_null = table_join.where(lambda row: row[classify_column] == None)

                if table_join_null:
                    print("Not everything joined. Table saved to tmp dir.")
                    table_join.to_csv('../tmp/table-not-joined.csv')
                
            print("saving joined table")
            table_join.exclude(
                ['geo2', 'geo2_2', 'geo2_3', 'county2', 'county2_2', 'county22', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm' ,'n', 'o', 'p' ,'q']
            ).to_csv(g_table_output)
        # First time through
        # Don't join tables
        else:
            print("file doesn't exist")
            copyfile(fp, g_table_output)
        
    except ValueError:
        return False

In [14]:
g_table_merge_fp = '../edits/prenatal/race/01-counties.csv'
fp = '../data/prenatal/rural-urban-defs.csv'
g_table_output = '../edits/prenatal/race/02-counties-urbanization.csv'
classify_column = 'county'
columns = ['county', 'white_percent	white_number', 'black_percent', 'black_number', 'hispanic_percent', 'hispanic_number', 'other_percent', 'other_number', 'Urbanization Status', 'Border Status', 'geo']

join_tables(fp, classify_column, columns)

file exists
saving joined table
